In [2]:
import pandas as pd
import numpy as np
from sklearn.cross_validation import train_test_split
import xgboost as xgb
import operator
import matplotlib
matplotlib.use("Agg") #Needed to save figures
import matplotlib.pyplot as plt

In [19]:
print("Load the training, test and store data using pandas")
types = {'CompetitionOpenSinceYear': np.dtype(int),
         'CompetitionOpenSinceMonth': np.dtype(int),
         'StateHoliday': np.dtype(str),
         'Promo2SinceWeek': np.dtype(int),
         'SchoolHoliday': np.dtype(float),
         'PromoInterval': np.dtype(str)}
train = pd.read_csv("../data/train.csv", parse_dates=[2], dtype=types)
test = pd.read_csv("../data/test.csv", parse_dates=[3], dtype=types)
store = pd.read_csv("../data/store.csv")

train.fillna(1, inplace=True)
test.fillna(1, inplace=True)
train = train[train["Open"] != 0]
train = train[train["Sales"] > 0]

train = pd.merge(train, store, on='Store')

Load the training, test and store data using pandas


In [31]:
def build_features_store(store, train):
#     store['Sales25th'] = 0
#     store['Sales50th'] = 0
#     store['Sales75th'] = 0
#     store['SalesMedian'] = 0
#     store['Customer25th'] = 0
#     store['Customer50th'] = 0
#     store['Customer75th'] = 0
#     store['CustomerMedian'] = 0

#     for store_id in train.Store.unique():
#         Sales25th = train[train.Store == store_id].Sales.quantile(0.25)
#         Sales50th = train[train.Store == store_id].Sales.quantile(0.50)
#         Sales75th = train[train.Store == store_id].Sales.quantile(0.75)
#         SalesMedian = train[train.Store == store_id].Sales.median()
#         Customer25th = train[train.Store == store_id].Customers.quantile(0.25)
#         Customer50th = train[train.Store == store_id].Customers.quantile(0.50)
#         Customer75th = train[train.Store == store_id].Customers.quantile(0.75)
#         CustomerMedian = train[train.Store == store_id].Customers.median()
    
#         store.loc[store.Store == store_id, 'Sales25th'] = Sales25th
#         store.loc[store.Store == store_id, 'Sales50th'] = Sales50th
#         store.loc[store.Store == store_id, 'Sales75th'] = Sales75th
#         store.loc[store.Store == store_id, 'SalesMedian'] = SalesMedian
    train = train.copy()
    train['Year'] = train.Date.dt.year
    train['Month'] = train.Date.dt.month
    train['Day'] = train.Date.dt.day
    train['DayOfWeek'] = train.Date.dt.dayofweek
    train['WeekOfYear'] = train.Date.dt.weekofyear
    for day in train.Day.unique():
        store['MeanSalesPerDay' + str(day)]
    
build_features_store(store, train)
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval,...,PerStoreMeanSalesPerDay14,PerStoreMeanSalesPerDay13,PerStoreMeanSalesPerDay11,PerStoreMeanSalesPerDayOfWeek4,PerStoreMeanSalesPerDayOfWeek3,PerStoreMeanSalesPerDayOfWeek2,PerStoreMeanSalesPerDayOfWeek1,PerStoreMeanSalesPerDayOfWeek0,PerStoreMeanSalesPerDayOfWeek5,PerStoreMeanSalesPerDayOfWeek6
0,1,c,a,1270,9,2008,0,NaN,NaN,NaN,...,4650.76,4426.037037,4236.750000,4726.480620,4457.838710,4555.712121,4685.626866,5177.968750,4942.970149,NaN
1,2,a,a,570,11,2007,1,13,2010,"Jan,Apr,Jul,Oct",...,4681.88,4542.740741,4281.785714,4669.589147,4967.472441,5834.946970,5369.201493,6061.953125,2873.104478,NaN
2,3,a,a,14130,12,2006,1,14,2011,"Jan,Apr,Jul,Oct",...,6354.84,6332.111111,6007.107143,7193.203125,6936.435484,7132.181818,7644.843284,8338.390625,4467.984962,NaN
3,4,c,c,620,9,2009,0,NaN,NaN,NaN,...,9307.16,9133.777778,8639.035714,9470.410853,9072.874016,8919.984848,9409.902985,10851.578125,10113.447761,NaN
4,5,a,a,29910,4,2015,0,NaN,NaN,NaN,...,4306.20,4383.296296,3975.535714,4858.054688,4747.960317,5212.084615,5109.694030,6156.488189,2079.104478,NaN
